# PySpark Vectorized UDFs using Arrow
Ref : https://bryancutler.github.io/


Using Arrow, it is possible to perform vectorized evaluation of Python UDFs that will accept one or more `Pandas.Series` as input and return a single `Pandas.Series` of equal length. Using vectorized functions will offer a performance boost over the current way PySpark evaluates using a loop that iterates over 1 row at a time.

## Where to get it
This functionality is currently pending review and has not yet been merged into Spark, see [SPARK-21404](https://issues.apache.org/jira/browse/SPARK-21404). Until then, a patch for this can be downloaded from the branch in the PR [here](https://patch-diff.githubusercontent.com/raw/apache/spark/pull/18659.diff).

## PySpark API
A new API has been added in pyspark to declare a vectorized UDF.  As with normal UDFs you can wrap a function or use a decorator:

```python
# Wrap the function "func"
pandas_udf(func, DoubleType())

# Use a decorator
@pandas(returnType=DoubleType())
def func(x):
    # do something with "x" (pandas.Series) and return "y" (also a pandas.Series)
    return y
```

## Example Usage
Let's go through a simple example with first evaluating a UDF without vectorization, then the same UDF with vectorization enabled. This will define some sample data:

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("PySpark_Vectorized_UDFs")\
    .getOrCreate()

In [2]:
from pyspark.sql.functions import col, udf, mean, rand
from pyspark.sql.types import *

df = spark.range(1 << 24, numPartitions=16).toDF("id") \
        .withColumn("p1", rand()).withColumn("p2", rand())

### First define the function *without vectorization*

In [3]:
from math import log, exp

def my_func(p1, p2):
    w = 0.5
    return exp(log(p1) + log(p2) - log(w))

and evaluate it as a UDF (using `filter()` to force evaluation)

In [4]:
my_udf = udf(my_func, DoubleType())

result = df.withColumn("p", my_udf(col("p1"), col("p2")))

%timeit result.filter("p < 1.0").count()

17.9 s ± 1.28 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Now define the function *with vectorization*

In [5]:
from numpy import log, exp

def my_func(p1, p2):
    w = 0.5
    return exp(log(p1) + log(p2) - log(w))

and evaluate the UDF again, this time making use of Arrow to evaluate `my_func` with `p1` and `p2` as `Pandas.Series`, which will also cause the expression to return a `Pandas.Series` of the same size.

NOTE: Spark will not accept Numpy types as return values, which is why we need to redefine the function.  This is an known issue from [SPARK-12157](https://issues.apache.org/jira/browse/SPARK-12157)

In [6]:
from pyspark.sql.functions import pandas_udf

my_udf = pandas_udf(my_func, DoubleType())

result = df.withColumn("p", my_udf(col("p1"), col("p2")))

%timeit result.filter("p < 1.0").count()

5.47 s ± 490 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Make better use of Pandas and Numpy

Since the inputs to your UDF are `Pandas.Series`, you can use Pandas and Numpy operations on the data and also return a series or numpy array. For example, say we want to draw samples from a random distribution for data points with a specific label.

In [7]:
import numpy as np
import pandas as pd

df = spark.range(1 << 20).toDF("id") \
        .selectExpr("(id % 3) AS label")

def sample(label):
    """ 
    Sample selected data from a Poisson distribution
    :param label: Pandas.Series of data labels
    """

    # use numpy to initialze an empty array
    p = pd.Series(np.zeros(len(label)))

    # use pandas to select data matching label "0"
    idx0 = label == 0

    # sample from numpy and assign to the selected data
    p[idx0] = np.random.poisson(7, len(idx0))

    # return the pandas series
    return p

sample_udf = pandas_udf(sample, DoubleType())

result = df.withColumn("counts", sample_udf(col("label")))
result.show(n=7)

+-----+------+
|label|counts|
+-----+------+
|    0|   7.0|
|    1|   0.0|
|    2|   0.0|
|    0|   7.0|
|    1|   0.0|
|    2|   0.0|
|    0|   5.0|
+-----+------+
only showing top 7 rows

